In [2]:
import ibis

ibis.options.interactive = True

In [3]:
conn = ibis.sqlite.connect('/tmp/geography.db')

In [4]:
countries = conn.table('countries')
countries['name', 'continent', 'area_km2', 'population'].limit(10)

,name,continent,area_km2,population
0,Andorra,EU,468.0,84000
1,United Arab Emirates,AS,82880.0,4975593
2,Afghanistan,AS,647500.0,29121286
3,Antigua and Barbuda,NA,443.0,86754
4,Anguilla,NA,102.0,13254
5,Albania,EU,28748.0,2986952
6,Armenia,AS,29800.0,2968000
7,Netherlands Antilles,NA,960.0,300000
8,Angola,AF,1246700.0,13068161
9,Antarctica,AN,14000000.0,0


In [6]:
continent_name = (
    countries['continent']
    .case()
    .when('NA', 'North America')
    .when('SA', 'South America')
    .when('EU', 'Europe')
    .when('AF', 'Africa')
    .when('AS', 'Asia')
    .when('OC', 'Oceania')
    .when('AN', 'Antarctica')
    .else_('Unknown')
    .end()
    .name('continent_name'))

continent_name

,continent_name
0,Europe
1,Asia
2,Asia
3,North America
4,North America
...,...
247,Asia
248,Africa
249,Africa
250,Africa


In [7]:
type(continent_name)

ibis.expr.types.strings.StringColumn

In [9]:
countries['name', continent_name, 'area_km2', 'population'].limit(10)

,name,continent_name,area_km2,population
0,Andorra,Europe,468.0,84000
1,United Arab Emirates,Asia,82880.0,4975593
2,Afghanistan,Asia,647500.0,29121286
3,Antigua and Barbuda,North America,443.0,86754
4,Anguilla,North America,102.0,13254
5,Albania,Europe,28748.0,2986952
6,Armenia,Asia,29800.0,2968000
7,Netherlands Antilles,North America,960.0,300000
8,Angola,Africa,1246700.0,13068161
9,Antarctica,Antarctica,14000000.0,0


In [11]:
countries[
    'name', continent_name.name('continent'), 'area_km2', 'population'
].limit(10)

,name,continent,area_km2,population
0,Andorra,Europe,468.0,84000
1,United Arab Emirates,Asia,82880.0,4975593
2,Afghanistan,Asia,647500.0,29121286
3,Antigua and Barbuda,North America,443.0,86754
4,Anguilla,North America,102.0,13254
5,Albania,Europe,28748.0,2986952
6,Armenia,Asia,29800.0,2968000
7,Netherlands Antilles,North America,960.0,300000
8,Angola,Africa,1246700.0,13068161
9,Antarctica,Antarctica,14000000.0,0


In [12]:
(countries
 .group_by(continent_name)
 .aggregate(countries['population'].sum().name('total_population')))

,continent_name,total_population
0,Africa,1021238685
1,Antarctica,170
2,Asia,4130584841
3,Europe,750724554
4,North America,540204371
5,Oceania,36067549
6,South America,400143568


In [14]:
total_population = (
    countries['population'].sum().name('total_population'))
total_population

6878963738

In [18]:
(countries
 .group_by('continent')
 .aggregate(
     [countries['population'].mean().name('mean_pop'),
      countries['area_km2'].mean().name('mean_area')]))

,continent,mean_pop,mean_area
0,AF,1.760756e+07,5.234534e+05
1,AN,3.400000e+01,2.802439e+06
2,AS,8.099186e+07,6.196685e+05
3,EU,1.390231e+07,4.293017e+05
4,NA,1.286201e+07,5.836313e+05
5,OC,1.288127e+06,3.044157e+05
6,SA,2.858168e+07,1.272751e+06


In [19]:
gdp = conn.table('gdp')
gdp.limit(10)

,country_code,year,value
0,ABW,1986,4.054634e+08
1,ABW,1987,4.876025e+08
2,ABW,1988,5.964236e+08
3,ABW,1989,6.953044e+08
4,ABW,1990,7.648871e+08
5,ABW,1991,8.721387e+08
6,ABW,1992,9.584632e+08
7,ABW,1993,1.082980e+09
8,ABW,1994,1.245688e+09
9,ABW,1995,1.320475e+09


In [20]:
gdp['year'].min(), gdp['year'].max()

(1960, 2017)

In [22]:
countries['iso_alpha2', 'iso_alpha3', 'iso_numeric', 'fips', 'name']

,iso_alpha2,iso_alpha3,iso_numeric,fips,name
0,AD,AND,20,AN,Andorra
1,AE,ARE,784,AE,United Arab Emirates
2,AF,AFG,4,AF,Afghanistan
3,AG,ATG,28,AC,Antigua and Barbuda
4,AI,AIA,660,AV,Anguilla
...,...,...,...,...,...
247,YE,YEM,887,YM,Yemen
248,YT,MYT,175,MF,Mayotte
249,ZA,ZAF,710,SF,South Africa
250,ZM,ZMB,894,ZA,Zambia


In [23]:
countries_gdp = countries.inner_join(
    gdp, predicates=countries.iso_alpha3 == gdp.country_code)
countries_gdp.limit(10)

,iso_alpha2,iso_alpha3,iso_numeric,fips,name,capital,area_km2,population,continent,country_code,year,value
0,AD,AND,20,AN,Andorra,Andorra la Vella,468.0,84000,EU,AND,1970,7.861921e+07
1,AD,AND,20,AN,Andorra,Andorra la Vella,468.0,84000,EU,AND,1971,8.940982e+07
2,AD,AND,20,AN,Andorra,Andorra la Vella,468.0,84000,EU,AND,1972,1.134082e+08
3,AD,AND,20,AN,Andorra,Andorra la Vella,468.0,84000,EU,AND,1973,1.508201e+08
4,AD,AND,20,AN,Andorra,Andorra la Vella,468.0,84000,EU,AND,1974,1.865587e+08
5,AD,AND,20,AN,Andorra,Andorra la Vella,468.0,84000,EU,AND,1975,2.201272e+08
6,AD,AND,20,AN,Andorra,Andorra la Vella,468.0,84000,EU,AND,1976,2.272810e+08
7,AD,AND,20,AN,Andorra,Andorra la Vella,468.0,84000,EU,AND,1977,2.540202e+08
8,AD,AND,20,AN,Andorra,Andorra la Vella,468.0,84000,EU,AND,1978,3.080089e+08
9,AD,AND,20,AN,Andorra,Andorra la Vella,468.0,84000,EU,AND,1979,4.115783e+08


In [24]:
gdp_2017 = gdp.filter(gdp.year == 2017)
gdp_2017

,country_code,year,value
0,ABW,2017,2.700559e+09
1,AFG,2017,2.019176e+10
2,AGO,2017,1.221238e+11
3,ALB,2017,1.302506e+10
4,AND,2017,3.013387e+09
...,...,...,...
242,XKX,2017,7.227700e+09
243,YEM,2017,2.681870e+10
244,ZAF,2017,3.495541e+11
245,ZMB,2017,2.586814e+10


In [26]:
countries_gdp = countries.inner_join(
    gdp_2017, predicates=countries.iso_alpha3 == gdp_2017.country_code)
countries_gdp.limit(5)

,iso_alpha2,iso_alpha3,iso_numeric,fips,name,capital,area_km2,population,continent,country_code,year,value
0,AW,ABW,533,AA,Aruba,Oranjestad,193.0,71566,NA,ABW,2017,2.700559e+09
1,AF,AFG,4,AF,Afghanistan,Kabul,647500.0,29121286,AS,AFG,2017,2.019176e+10
2,AO,AGO,24,AO,Angola,Luanda,1246700.0,13068161,AF,AGO,2017,1.221238e+11
3,AL,ALB,8,AL,Albania,Tirana,28748.0,2986952,EU,ALB,2017,1.302506e+10
4,AD,AND,20,AN,Andorra,Andorra la Vella,468.0,84000,EU,AND,2017,3.013387e+09
